In [38]:
import numpy as np
import numbers

In [39]:
#to find total no.of classes in the dataset
#dictionary will have class label and count of each label
def countClass(data):
    dictLabelCount = {}
    for row in data:
        label = row[-1]
        if label not in dictLabelCount:
            dictLabelCount[label] = 0
        dictLabelCount[label] += 1
    return dictLabelCount

In [40]:
#to compute the gini index for dataset 
def gini(data):
    counts = countClass(data)
    giniVal = 1
    for label in counts:
        prob = counts[label] / float(len(data))
        giniVal -= prob ** 2
    return giniVal

In [41]:
#dividing data based on attribute and attribute value
def partition(data, col, val):
    right = []
    left = []
    for row in data:
        if isinstance(val, numbers.Number):
            val = float(val)
            col= int(col)
            if row[col] >= val:
                right.append(row)
            else:
                left.append(row)
        else:
            if row[col] == val:
                right.append(row)
            else:
                left.append(row)
    return left, right

In [42]:
#computing gain using gini index
def bestGainSplit(data):
    bestGain , bestCol, bestVal = 0, 0, 0
    if len(data) == 0:
        return bestGain, bestCol, bestVal

    curGINI = gini(data)
    features = len(data[0]) - 1
        
    for col in range(features):
        values = set([row[col] for row in data])
        for val in values:
            leftClass, rightClass = partition(data, col, val)
            if isinstance(val, numbers.Number):
                val = float(val)
                col = int(col)
            if len(leftClass) == 0 or len(rightClass) == 0:
                continue
            total = (len(rightClass) + len(leftClass))
            probRight = float(len(rightClass) / total)
            probleft = float(len(leftClass) / total)
            #information gain calculation
            gain = curGINI - (probleft * gini(leftClass)) - (probRight * gini(rightClass))
        
            if gain >= bestGain:
                bestGain = gain
                bestCol = col
                bestVal = val
       
    return bestGain, bestCol, bestVal

In [43]:
#how a prticular node looks like
class TreeNode(object):
    def __init__(self,col,val, left, right, rightClass, leftClass):
        self.col = col
        self.val = val
        self.left = left
        self.right = right
        self.rightClass=rightClass
        self.leftClass=leftClass

In [44]:
#determining class of node after a perfect split
def updateNode(data,col, val):
    rightClass=0
    leftClass=0
    for row in data:
        if int(row[-1])==1:
            rightClass +=1
        else:
            leftClass +=1
    return TreeNode(col, val, None, None,rightClass, leftClass)

In [45]:
#building decision tree
def buildDecisionTree(data):
    gain, col, val = bestGainSplit(data)
    if gain == 0:
        return updateNode(data,None,None)
    leftClass, rightClass = partition(data, col, val)
    if isinstance(val, numbers.Number):
        val = float(val)
        col = int(col)
    node = TreeNode(col,val, None, None,-1,-1)
    node.left = buildDecisionTree(leftClass)
    node.right = buildDecisionTree(rightClass)
    return node

In [46]:
def displayTree(root, tab):
    tab+="      "
    if (root.left == None) and (root.right == None):
        if root.rightClass >= root.leftClass:
            print(tab + "   ->(Class 1)" )
        else:
            print(tab + "   ->(Class 0)" )
        return
    print(tab  + "SPLIT" + ": " + str(root.val))
    print( tab + '   ->left:')
    displayTree(root.left, tab )
    print( tab + '   ->Right:')
    displayTree(root.right, tab )

In [47]:
#predicting class label for test data
def predictClass(root,data):
    if root.left == None and root.right==None:
        if (root.rightClass ==-1) and (root.leftClass == -1):
            print("")
        if(root.rightClass >= root.leftClass):
            return 1
        else:
            return 0
    if isinstance(root.val, numbers.Number):
        if(data[root.col] >= root.val):
            return predictClass(root.right,data)
        else:
            return predictClass(root.left,data)
    else:
        if data[root.col] == root.val:
            return predictClass(root.right,data)
        else:
            return predictClass(root.left,data)

In [48]:
#dividing data into testing and training
def split_data(data, iteration, no_of_folds):
    batch = len(data) / no_of_folds
    training_first = data[:int((iteration - 1) * batch)]
    testing_data = data[int((iteration - 1) * batch):int(iteration * batch)]
    training_last = data[int(iteration * batch):]
    training_data = np.concatenate((training_first, training_last), axis=0)
    return training_data, testing_data

In [49]:
#reading input file
input_file = "project3_dataset1.txt"
data = np.genfromtxt(input_file, dtype=None)
data = np.array(data)

fold = 10

totalAccuracy = []
totalPrecision = []
totalRecall = []
totalFMeasure = []

for i in range(fold):
    train_data, test_data = split_data(data, i + 1, 10)
    root = None
    root = buildDecisionTree(train_data)
    
    #printing tree
    #displayTree(root,"")
    
    
    test_label = []
    for row in test_data:
        test_label.append(int(row[-1]))

    predictedLabels = []
    for tdata in test_data:
        label = predictClass(root,tdata)
        predictedLabels.append(int(label))
    
    truePositive = 0
    trueNegative = 0
    falseNegative = 0
    falsePositive = 0
    
    for j in range(len(test_label)):
        if(test_label[j] == predictedLabels[j]):
            if predictedLabels[j]==0:
                trueNegative +=1
            else:
                truePositive +=1
        else:
            if(predictedLabels[j]==0):
                falseNegative +=1
            else:
                falsePositive +=1
    Accuracy = float(truePositive+trueNegative)/(truePositive+trueNegative+falseNegative+falsePositive)
    Accuracy *= 100
    Precision = float(truePositive/(truePositive+falsePositive))
    Precision *= 100
    Recall = float(truePositive/(truePositive+falseNegative))
    Recall *= 100
    FMeasure = float((2*truePositive)/((2*truePositive)+falseNegative+falsePositive))        

    totalAccuracy.append(Accuracy)
    totalPrecision.append(Precision)
    totalRecall.append(Recall)
    totalFMeasure.append(FMeasure)
    #print("Iteration", i + 1)

print("averageAccuracy  : "+str(np.sum(totalAccuracy)/fold))
print("averagePrecision  : "+str(np.sum(totalPrecision)/fold))
print("averageRecall  : "+str(np.sum(totalRecall)/fold))
print("averageFMeasure  : "+str(np.sum(totalFMeasure)/fold))  

averageAccuracy  : 91.9204260651629
averagePrecision  : 89.17466930052743
averageRecall  : 89.02665450491538
averageFMeasure  : 0.8885181069233683
